In [11]:
from __future__ import print_function
import argparse
from math import log10
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from data import get_training_set, get_test_set
import pdb
import socket
import time

from pmpanet_x2 import Net as PMBAX2
from pmpanet_x4 import Net as PMBAX4
from pmpanet_x8 import Net as PMBAX8
from pmpanet_x16 import Net as PMBAX16

In [12]:
# # Training settings
# parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
# parser.add_argument('--upscale_factor', type=int, default=8, help="super resolution upscale factor")
# parser.add_argument('--batchSize', type=int, default=8, help='training batch size')
# parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
# parser.add_argument('--nEpochs', type=int, default=400, help='number of epochs to train for')
# parser.add_argument('--snapshots', type=int, default=30, help='Snapshots')
# parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=0.01')
# parser.add_argument('--gpu_mode', type=bool, default=False)
# parser.add_argument('--threads', type=int, default=10, help='number of threads for data loader to use')
# parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
# parser.add_argument('--gpus', default=2, type=float, help='number of gpu')
# parser.add_argument('--data_dir', type=str, default='/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/data/')
# parser.add_argument('--data_augmentation', type=bool, default=True)
# parser.add_argument('--hr_train_dataset', type=str, default='train_x8_hr/')
# parser.add_argument('--rgb_train_dataset', type=str, default='train_x8_rgb/')
# parser.add_argument('--train_dataset', type=str, default='train_x8/')
# parser.add_argument('--model_type', type=str, default='PMBAX8')
# parser.add_argument('--patch_size', type=int, default=32, help='Size of cropped HR image')
# parser.add_argument('--pretrained_sr', default='Server1PMPA4x43ct_epoch_299_x4.pth', help='sr pretrained base model')
# parser.add_argument('--pretrained', type=bool, default=False)
# parser.add_argument('--save_folder', default='./weights/x8/', help='Location to save checkpoint models')
# parser.add_argument('--prefix', default='dbpn', help='Location to save checkpoint models')
# opt = parser.parse_args()
# gpus_list = range(opt.gpus)
# hostname = str(socket.gethostname())
# print(opt)


In [13]:
parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
parser.add_argument('--upscale_factor', type=int, default=8, help="super resolution upscale factor")
parser.add_argument('--batchSize', type=int, default=8, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--nEpochs', type=int, default=400, help='number of epochs to train for')
parser.add_argument('--snapshots', type=int, default=30, help='Snapshots')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=0.01')
parser.add_argument('--gpu_mode', type=bool, default=False)
parser.add_argument('--threads', type=int, default=10, help='number of threads for data loader to use')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--gpus', default=2, type=float, help='number of gpu')
parser.add_argument('--data_dir', type=str, default='/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/data/')
parser.add_argument('--data_augmentation', type=bool, default=True)
parser.add_argument('--hr_train_dataset', type=str, default='train_x8_hr/')
parser.add_argument('--rgb_train_dataset', type=str, default='train_x8_rgb/')
parser.add_argument('--train_dataset', type=str, default='train_lr/')
parser.add_argument('--model_type', type=str, default='PMBAX8')
parser.add_argument('--patch_size', type=int, default=32, help='Size of cropped HR image')
parser.add_argument('--pretrained_sr', default='Server1PMPA4x43ct_epoch_299_x4.pth', help='sr pretrained base model')
parser.add_argument('--pretrained', type=bool, default=False)
parser.add_argument('--save_folder', default='./weights/x8/', help='Location to save checkpoint models')
parser.add_argument('--prefix', default='dbpn', help='Location to save checkpoint models')
opt = parser.parse_args("")
gpus_list = range(opt.gpus)
hostname = str(socket.gethostname())
print(opt)



Namespace(batchSize=8, data_augmentation=True, data_dir='/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/data/', gpu_mode=False, gpus=2, hr_train_dataset='train_x8_hr/', lr=0.0001, model_type='PMBAX8', nEpochs=400, patch_size=32, prefix='dbpn', pretrained=False, pretrained_sr='Server1PMPA4x43ct_epoch_299_x4.pth', rgb_train_dataset='train_x8_rgb/', save_folder='./weights/x8/', seed=123, snapshots=30, testBatchSize=1, threads=10, train_dataset='train_lr/', upscale_factor=8)


In [ ]:
def read_data_files():
    # load 1 oblique image for test of hyperparameter
_rgb_files = '/home/beast2020/Desktop/mingchi/EPFL_10_newsample_ob/rgb/EPFL-2020-09-24-planned_01122_100_0003_0334.png'
_dist_files = '/home/beast2020/Desktop/mingchi/EPFL_10_newsample_ob/dist/EPFL-2020-09-24-planned_01122_100_0003_0334.dat'


guide_imgs = []
target_imgs= []
rgb_img = imageio.imread(_rgb_files)
guide_imgs.append(rgb_img)
dist_img = torch.load(_dist_files)
dist_img = dist_img.detach().cpu().numpy()
target_imgs.append(dist_img)
        
guide_imgs = np.asarray(guide_imgs)
guide_imgs = np.einsum('ijkl->iljk', guide_imgs)        
target_imgs = np.asarray(target_imgs)
    return input_rgb, input_lr, target

In [26]:
train(0)

/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/data/train_x8_hr/2.png (1344, 1088)


UnboundLocalError: Caught UnboundLocalError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/dataset.py", line 179, in __getitem__
UnboundLocalError: local variable 'input_lr' referenced before assignment


In [25]:
def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):

        input_rgb, input_lr, target = Variable(batch[0]), Variable(batch[1]), Variable(batch[2])
        print(input_rgb.shape)
        print(input_lr.shape)
        print(target.shape)
        if cuda:
            input_rgb = input_rgb.cuda(gpus_list[1])
            input_lr = input_lr.cuda(gpus_list[1])
            target = target.cuda(gpus_list[1])

        optimizer.zero_grad()
        t0 = time.time()

        loss = criterion(model(input_rgb, input_lr), target)
        t1 = time.time()
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        avg_loss = epoch_loss / len(training_data_loader)
        loss_list.append(avg_loss)

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, iteration, len(training_data_loader), loss.item(), (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))


def test():
    avg_psnr = 0
    for batch in testing_data_loader:

        input_rgb, input_lr, target = Variable(batch[0]), Variable(batch[1]), Variable(batch[2])
        if cuda:
            input_rgb = input_rgb.cuda(gpus_list[1])
            input_lr = input_lr.cuda(gpus_list[1])
            target = target.cuda(gpus_list[1])

        prediction = model(input_rgb, input_lr)
        mse = criterion(prediction, target)
        psnr = 10 * log10(1 / mse.item())
        avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))

def print_network(net):
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)

def checkpoint(epoch):
    model_out_path = opt.save_folder+opt.train_dataset+hostname+opt.model_type+opt.prefix+"_epoch_{}.pth".format(epoch)
    torch.save(model.state_dict(), model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))



In [15]:
# gpu setting
cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if cuda:
    torch.cuda.manual_seed(opt.seed)

In [21]:
# load data


print('===> Loading datasets')
train_set = get_training_set(opt.data_dir, opt.train_dataset, opt.hr_train_dataset, opt.rgb_train_dataset, opt.upscale_factor, opt.patch_size, opt.data_augmentation)


training_data_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)
# train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

===> Loading datasets


In [22]:
train_set

In [23]:

print('===> Building model ', opt.model_type)
if opt.model_type == 'PMBAX8':
    model = PMBAX8(num_channels=1, base_filter=64,  feat = 256, num_stages=3, scale_factor=opt.upscale_factor)
else:
    model = PMBAX8(num_channels=1, base_filter=64,  feat = 256, num_stages=5, scale_factor=opt.upscale_factor)
    
#model = torch.nn.DataParallel(model, device_ids=gpus_list)
criterion = nn.L1Loss()
#criterion = nn.MSELoss()

print('---------- Networks architecture -------------')
print_network(model)
print('----------------------------------------------')

if opt.pretrained:
    model_name = os.path.join(opt.save_folder + opt.pretrained_sr)
    if os.path.exists(model_name):
        pretrained_dict = torch.load(model_name, map_location=lambda storage, loc: storage)
        model_dict = model.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)


        #model= torch.load(model_name, map_location=lambda storage, loc: storage)
        # model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
        print('************************************Pre-trained SR model is loaded.************************************')
###############


===> Building model  PMBAX8


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


---------- Networks architecture -------------
Net(
  (feat0): ConvBlock(
    (conv): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat1): ConvBlock(
    (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat_color0): ConvBlock(
    (conv): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat_color1): ConvBlock(
    (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat_color2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat_color3): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (m1): MultiViewBlock1(
    (dilaconv1): DilaConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [ ]:
if cuda:
    model = model.cuda(gpus_list[1])
    criterion = criterion.cuda(gpus_list[1])

optimizer = optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)


In [20]:

loss_list = []
for epoch in range(1, opt.nEpochs + 1):
    train(epoch)

    if (epoch+1) % 10 == 0:
        plt.plot(loss_list, linewidth=5)
        plt.title('Loss Table', fontsize=24)
        plt.xlabel('epoch', fontsize=14)
        plt.ylabel('loss', fontsize=14)
        plt.tick_params(axis='both', labelsize=14)
        plt.savefig('PMBAX8.png')
    if (epoch+1) == 100:
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 10.0
        print('Learning rate decay: lr={}'.format(optimizer.param_groups[0]['lr']))

    if (epoch+1) == 200:
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 10.0
        print('Learning rate decay: lr={}'.format(optimizer.param_groups[0]['lr']))

    if (epoch+1) % (opt.snapshots) == 0:
        checkpoint(epoch)

/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/data/train_x8_hr/1.png (1344, 1088)


UnboundLocalError: Caught UnboundLocalError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/felicity/Desktop/Super-resolution-drones-depth-images/PBMANet_TOPO/dataset.py", line 179, in __getitem__
    input_lr,input_rgb,target, _ = augment(input_lr,input_rgb,target)
UnboundLocalError: local variable 'input_lr' referenced before assignment
